## Descrição da base de dados

Este relatório tem como objetivo descrever a base de dados adotada para a produção das demais análises e experimentos disponibilizados neste [repositório](https://github.com/alvesmatheus/reconhecimento-temas-comissoes). Para sua produção, serão utilizadas as seguintes bibliotecas:

- **[Altair](https://altair-viz.github.io/)**, para visualização dos dados.
- **[Pandas](https://pandas.pydata.org/)**, para manipulação e apresentação tabular dos dados.

In [1]:
import altair as alt
import pandas as pd

Essa base de dados foi construída a partir de raspagem de dados, tendo como alvo as páginas do [Portal da Câmara dos Deputados](https://www.camara.leg.br/) onde estão disponibilizadas as notas taquigráficas de sessões das comissões da Câmara dos Deputados realizadas de 1995 a 2020.

Contudo, este relatório não se concentra em descrever diretamente as transcrições extraídas, mas sim as características dos eventos a partir dos quais elas foram produzidas. Estes metadados também estão públicos no Portal da Câmara dos Deputados. Todo o processo de raspagem desses dados pode ser reproduzido com o uso do código disponível neste [repositório](https://github.com/alvesmatheus/fala-camarada).

O conjunto de metadados citado anteriormente contém **19.302 registros** e tem suas características descritas a seguir. Além disso, uma amostra desse conjunto de dados está apresentada abaixo, visando auxiliar na compreensão de sua estrutura.

| **Nome da Variável** | **Descrição** |
| :---: | :--- |
| **id_evento** | Identificador único para o evento. |
| **categoria_evento** | Categoria do evento. Permite identificar, por exemplo, as sessões em que ocorrem audiências públicas ou eleições. |
| **comissao** | Nome da comissão da Câmara dos Deputados a que o evento está associado. |
| **categoria_comissao** | Categoria de comissão definida a partir do nome da comissão da Câmara dos Deputados a que o evento está associado. |
| **data** | A data, no formato `DD/MM/YYYY`, em que o evento foi realizado. |
| **horario** | O horário, no formato `HH:MM`, em que o evento foi realizado. |

Ainda, uma coluna não-nativa do conjunto de dados (chamada **`ano`**) será criada para facilitar a compreensão da distribuição dos eventos ao longo do tempo.

In [2]:
DADOS_AGENDA_COMISSOES = '../data/sumario-transcricoes.csv'

agenda_comissoes = pd.read_csv(DADOS_AGENDA_COMISSOES)
agenda_comissoes['ano'] = agenda_comissoes['data'].apply(lambda x : int(x[-4:]))

agenda_comissoes.sample(10)

id_evento                 categoria_evento  \
18522     55726  Audiência Pública com Convidado   
11424   1329/09                           Outros   
18755     56842  Audiência Pública com Convidado   
8992    0870/06  Audiência Pública com Convidado   
11666   1751/09  Audiência Pública com Convidado   
10228   0200/08  Audiência Pública com Convidado   
14431   1494/13  Audiência Pública com Convidado   
14787   2263/13  Audiência Pública com Convidado   
12896   0872/11  Audiência Pública com Convidado   
8169    1190/05  Audiência Pública com Convidado   

                                                comissao  \
18522            Comissão de Direitos Humanos e Minorias   
11424                                     Outros Eventos   
18755            Comissão de Seguridade Social e Família   
8992              Conselho de Ética e Decoro Parlamentar   
11666            Comissão de Seguridade Social e Família   
10228             CPI - Escutas Telefônicas Clandestinas   
14431  Comissão de Constituição e Justiça e de Cidadania   
14787  Comissão de Meio Ambiente e Desenvolvimento Su...   
12896  Comissão de Agricultura, Pecuária, Abastecimen...   
8169                                  CPI - Biopirataria   

          categoria_comissao        data horario   ano  
18522    Comissão Permanente  04/06/2019   14:46  2019  
11424                 Outros  18/08/2009   14:00  2009  
18755    Comissão Permanente  22/08/2019   10:23  2019  
8992                  Outros  13/06/2006   10:39  2006  
11666    Comissão Permanente  13/10/2009   14:52  2009  
10228  Comissão de Inquérito  25/03/2008   14:45  2008  
14431    Comissão Permanente  24/09/2013   14:54  2013  
14787    Comissão Permanente  17/12/2013   14:36  2013  
12896    Comissão Permanente  28/06/2011   15:28  2011  
8169   Comissão de Inquérito  18/08/2005   10:19  2005

#### Funções utilitárias

In [3]:
def calcula_distribuicao_frequencia(dataframe, coluna):
    frequencias = dataframe[coluna].value_counts(dropna=False)

    return pd.DataFrame({
        coluna: frequencias.index,
        'frequencia': frequencias.values
    })

### Distribuições de frequência

Inicialmente, o conjunto de dados descrito anteriormente será avaliado a partir das distribuições de frequência de suas características. Desta forma, o gráfico a seguir apresenta a distribuição de frequência dos eventos quanto à categoria de comissão da Câmara dos Deputados a que estão associados.

In [4]:
dist_categoria_comissao = calcula_distribuicao_frequencia(
    dataframe = agenda_comissoes,
    coluna = 'categoria_comissao'
)

alt.Chart(
    dist_categoria_comissao
).mark_bar().encode(
    x = alt.X('frequencia',
              title = 'Número de notas táquigráficas'),
    y = alt.Y('categoria_comissao',
              title = 'Categoria de comissão',
              sort = '-x'),
).properties(
    title = 'Notas taquigráficas por categoria de comissão (1995 - 2020)',
    height = 400,
    width = 600
)

alt.Chart(...)

Como é possível percerber, os eventos associados a comissões permanentes da Câmara dos Deputados são responsáveis por, aproximadamente, 8 mil dos mais de 19 mil registros do conjunto de dados. Enquanto isso, dentre as categorias restantes, nenhuma alcança sequer metade dessa quantidade. 

Conhecendo essa distribuição e sabendo que as demais análises realizadas usando esta base de dados se concentrarão nas comissões permanentes da Câmara dos Deputados, a partir desse ponto serão utilizados apenas os registros cuja categoria de comissão tenha `Comissão Permanente` como valor.

Desta forma, considerando esse subconjunto dos dados, o gráfico a seguir apresenta a distribuição de frequência dos eventos quanto às suas categorias.

In [5]:
agenda_comissoes_permanentes = agenda_comissoes[agenda_comissoes['categoria_comissao'] == 'Comissão Permanente']

dist_categoria_evento = calcula_distribuicao_frequencia(
    dataframe = agenda_comissoes_permanentes,
    coluna = 'categoria_evento'
)

alt.Chart(
    dist_categoria_evento
).mark_bar().encode(
    x = alt.X('frequencia',
              title = 'Número de notas táquigráficas',
              scale = alt.Scale(type='log')),
    y = alt.Y('categoria_evento',
              title = 'Categoria de evento',
              sort = '-x'),
).properties(
    title = 'Notas taquigráficas de comissões permanentes por categoria de evento (1995 - 2020)',
    height = 600,
    width = 600
)

alt.Chart(...)

Considerando apenas as comissões permanentes da Câmara dos Deputados, foram identificadas **36 categorias de evento** cujas frequências são muito desiguais. Enquanto algumas categorias incluem alguns poucos eventos, outras incluem milhares deles.

Outro fator importante a se avaliar é a distribuição desses eventos ao longo do tempo. Assim, o gráfico abaixo apresenta a quantidade de eventos realizados por ano, no intervalo de 1995 a 2020.

In [6]:
dist_ano = calcula_distribuicao_frequencia(
    dataframe = agenda_comissoes_permanentes,
    coluna = 'ano'
)

alt.Chart(
    dist_ano
).mark_bar().encode(
    x = alt.X('frequencia',
              title = 'Número de notas táquigráficas'),
    y = alt.Y('ano',
              title = 'Ano',
              type = 'nominal')
).properties(
    title = 'Notas taquigráficas de comissões permanentes por ano (1995 - 2020)',
    height = 600,
    width = 800
)

alt.Chart(...)

A partir deste gráfico, é perceptível uma tendência de crescimento ao longo do tempo no número de eventos das comissões permanentes da Câmara dos Deputados cujas notas taquigráficas são disponibilizadas publicamente. Outra tendência notável é que, de 1995 a 2006, os anos de eleições majoritárias (1998, 2002 e 2006) apresentam uma súbita queda no número de notas taquigráficas disponíveis quando comparados aos anos que lhes antecedem.

Também é importante destacar que, especialmente em decorrência da pandemia de COVID-19, o ano de 2020 não teve eventos de comissões permanentes cujas notas taquigráficas tenham sido disponibilizadas.

Por fim, o gráfico a seguir apresenta a distribuição de frequência dos eventos em relação às comissões permanentes da Câmara dos Deputados a que estão associados.

In [7]:
dist_comissao = calcula_distribuicao_frequencia(
    dataframe = agenda_comissoes_permanentes,
    coluna = 'comissao'
)

alt.Chart(
    dist_comissao
).mark_bar().encode(
    x = alt.X('frequencia',
              title = 'Número de notas táquigráficas'),
    y = alt.Y('comissao',
              title = '',
              sort = '-x',
              axis = alt.Axis(labelLimit = 0)),
).properties(
    title = 'Notas taquigráficas de comissões permanentes por comissão (1995 - 2020)',
    height = 600,
    width = 500
)

alt.Chart(...)

A quantidade de notas taquigráficas disponibilizadas também se mostra muito distinta entre as comissões da Câmara dos Deputados, porém, esse tipo de variação já era esperada e pode ser causada por múltiplos fatores. Entre eles, a criação de novas comissões permanentes pode ser acompanhanda (com alguma imprecisão) usando a quantidade de comissões permanentes distintas cujos eventos estão presentes nessa base de dados, conforme apresentado no gráfico abaixo.

In [8]:
comissoes_por_ano = agenda_comissoes_permanentes.groupby('ano')['comissao'].nunique()
dist_comissoes_por_ano = pd.DataFrame({
    'ano': comissoes_por_ano.index,
    'n_comissoes': comissoes_por_ano.values,
})

alt.Chart(
    dist_comissoes_por_ano
).mark_bar().encode(
    x = alt.X('n_comissoes',
              title = 'Número de comissões permanentes'),
    y = alt.Y('ano',
              title = 'Ano',
              type = 'nominal')
).properties(
    title = 'Número de comissões permanentes encontradas por ano (1995 - 2020)',
    height = 500,
    width = 800
)

alt.Chart(...)

## Definição da amostra de interesse

Considerando a inviabilidade de avaliar individualmente as notas taquigráficas, bem como as características apresentadas neste relatório, optou-se por selecionar uma amostra dos dados que seja aparentemente mais adequada às análises relacionadas aos temas discutidos nas comissões da Câmara dos Deputados.

Os eventos cujas notas taquigráficas serão selecionadas para compor essa amostra de interesse são aqueles associados a **comissões permanentes** da Câmara dos Deputados, que tenham sido realizados de **2009 a 2018** e que estejam incluídos em uma das **categorias de evento** listadas abaixo.

- Audiência Pública com Convidado
- Audiência Pública com Ministro
- Debates
- Fórum
- Fórum de Debate
- Grupo de Trabalho
- Reunião Extraordinária
- Reunião Ordinária
- Reunião Técnica
- Palestra
- Seminário
- Simpósio

Também não está descartada a possibilidade de que, em diferentes análises, essa amostra seja submetida a novos filtros.

>**Nota:** A seleção de um período de tempo reduzido (2009 - 2018) foi realizada exclusivamente para atender às limitações de memória RAM disponível para execução das análies e experimentos.

In [9]:
CARACTERISTICAS_ALVO = {
    'ano': [ano for ano in range(2009, 2019)],
    'categoria_comissao': ['Comissão Permanente'],
    'categoria_evento': ['Audiência Pública com Convidado', 'Audiência Pública com Ministro',
                         'Debates', 'Fórum', 'Fórum de Debate', 'Grupo de Trabalho',
                         'Reunião Extraordinária', 'Reunião Ordinária', 'Reunião Técnica',
                         'Palestra', 'Seminário', 'Simpósio']
}

agenda_eventos_alvo = agenda_comissoes

for coluna, valores in CARACTERISTICAS_ALVO.items():
    agenda_eventos_alvo = agenda_eventos_alvo[agenda_eventos_alvo[coluna].isin(valores)]

Assim, a amostra selecionada descreve **3.458 eventos** de comissões permanentes da Câmara dos Deputados, realizados entre 2009 e 2018 e cuja categoria tende a exprimir os temas abordados nestas comissões.

Posteriormente notou-se que **5 destes eventos não tinham suas notas taquigráficas disponíveis** nas páginas indicadas no [Portal da Câmara dos Deputados](https://www.camara.leg.br/), tampouco se conseguiu identificar outra forma de obtê-las. Essa observação levou à descoberta que, do conjunto de dados original, 463 eventos não tem suas notas taquigráficas realmente disponibilizadas, o que representa pouco mais de 2% do total de eventos.